In [2]:
import os
%pwd
os.chdir("../")
%pwd

'd:\\ML_Projects'

In [5]:
import pandas as pd

In [87]:
ROOT_DIR = "dataset/audio_classification/raw_data/"
CSV_FILE = "set_a.csv"


In [6]:
df=pd.read_csv(ROOT_DIR+CSV_FILE)
# Filter rows based on column: 'label'
df = df[df['label'].notna()]
audio_file_paths= df['fname']
labels=df['label']
class_list=labels.unique()

In [43]:
# Split data with stratification to ensure each class has 80% for training and 20% for testing
from sklearn.model_selection import train_test_split


train_audio, test_audio, train_labels, test_labels = train_test_split(
    audio_file_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

In [65]:
from pathlib import Path
import matplotlib.pyplot as plt
import torch
import torchaudio

In [64]:
def plot_specgram(waveform, sample_rate, file_path = 'test2.png'):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    
    fig, axes = plt.subplots( num_channels, 1)
    fig.set_size_inches(10, 10)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    plt.gca().set_axis_off()
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
    # plt.show(block=False)
    plt.savefig(file_path, bbox_inches='tight', pad_inches = 0)

In [86]:
def create_spectrograms(file_paths,labels,data_type):
    for i in range(len(file_paths)):
        try:
            print(f"counting: {i+1}/{len(file_paths)}")
            # Access the file path using iloc to handle the Series indexing
            full_audio_path = ROOT_DIR + file_paths[i]  # Access via iloc
            label = labels[i]  # Access the label similarly
            f=file_paths[i].split("/")[1]
            file_name_without_ext = os.path.splitext(os.path.basename(f))[0]
            target_path = Path(ROOT_DIR +f"{data_type}" +"/"+label + "/" + file_name_without_ext + ".png")
            target_path.parent.mkdir(parents=True, exist_ok=True)

            # extract class type from file, e.g. 
            data_waveform, sr = torchaudio.load(full_audio_path)
            # create spectrogram and save it
            plot_specgram(waveform=data_waveform, sample_rate=sr, file_path=target_path)
        except Exception as e:
            # Print an error message and continue to the next file
            print(f"Error processing file {full_audio_path}: {e}")

In [ ]:

# train data
create_spectrograms(train_audio.to_list(),train_labels.to_list(),"train")
# # test data
create_spectrograms(test_audio.to_list(),test_labels.to_list(),"test")

    